In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import catboost as cb
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

In [23]:
data = pd.read_excel("train.xlsx", skiprows=5)
data = data.fillna(0).iloc[:244]

In [24]:
named_cols = data.columns.tolist()

In [4]:
data[named_cols[:37]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 37 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   год                                      244 non-null    float64       
 1   неделя                                   244 non-null    int64         
 2   Начало нед                               244 non-null    datetime64[ns]
 3   Продажи, рубли                           244 non-null    float64       
 4   Продажи, упаковки                        244 non-null    float64       
 5   Продажи, рубли.1                         244 non-null    float64       
 6   раствор 0.01 % 150 мл N1, руб            244 non-null    float64       
 7   раствор  0.01 % 50 мл N1, руб            244 non-null    float64       
 8   раствор 0.01 % 500 мл N1, руб            244 non-null    float64       
 9   раствор 0.01 % 150 мл N1, упаковки       24

In [6]:
sns.pairplot(data[named_cols[2:37]])

In [35]:
data = data[named_cols[3:37]]

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 34 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Продажи, рубли                           244 non-null    float64
 1   Продажи, упаковки                        244 non-null    float64
 2   Продажи, рубли.1                         244 non-null    float64
 3   раствор 0.01 % 150 мл N1, руб            244 non-null    float64
 4   раствор  0.01 % 50 мл N1, руб            244 non-null    float64
 5   раствор 0.01 % 500 мл N1, руб            244 non-null    float64
 6   раствор 0.01 % 150 мл N1, упаковки       244 non-null    float64
 7   раствор  0.01 % 50 мл N1, упаковки       244 non-null    float64
 8   раствор 0.01 % 500 мл N1, упаковки       244 non-null    float64
 9   Дистрибуция Мирамистин                   244 non-null    float64
 10  Выписки                                  244 non-n

In [8]:
train, test = train_test_split(data,
                test_size=0.2,
                train_size=0.8,
                random_state=1,
                shuffle=True)

In [9]:
train_y = train[["Продажи, рубли"]]
train_x = train.drop(columns=["Продажи, рубли", "Продажи, упаковки"])
test_y = test[["Продажи, рубли"]]
test_x = test.drop(columns=["Продажи, рубли", "Продажи, упаковки"])

In [10]:
cat_train_pool = cb.Pool(train_x, 
                  train_y)

cat_test_pool = cb.Pool(test_x, 
                  test_y)

In [20]:
model = cb.CatBoostRegressor(iterations=5000, 
                          depth=10, 
                          learning_rate=0.1, 
                          loss_function='MAPE')

In [21]:
model.fit(cat_train_pool)
preds = model.predict(cat_test_pool)

0:	learn: 0.1732088	total: 15.9ms	remaining: 1m 19s
1:	learn: 0.1675866	total: 27.5ms	remaining: 1m 8s
2:	learn: 0.1610963	total: 37.4ms	remaining: 1m 2s
3:	learn: 0.1547411	total: 46.8ms	remaining: 58.5s
4:	learn: 0.1477706	total: 56.4ms	remaining: 56.3s
5:	learn: 0.1407245	total: 65ms	remaining: 54.1s
6:	learn: 0.1368181	total: 73.5ms	remaining: 52.4s
7:	learn: 0.1335291	total: 79ms	remaining: 49.3s
8:	learn: 0.1289827	total: 82.7ms	remaining: 45.9s
9:	learn: 0.1251827	total: 92.8ms	remaining: 46.3s
10:	learn: 0.1196096	total: 102ms	remaining: 46.4s
11:	learn: 0.1160107	total: 111ms	remaining: 46.3s
12:	learn: 0.1138630	total: 120ms	remaining: 46.1s
13:	learn: 0.1107156	total: 129ms	remaining: 46.1s
14:	learn: 0.1064849	total: 138ms	remaining: 46s
15:	learn: 0.1033148	total: 147ms	remaining: 45.7s
16:	learn: 0.0991172	total: 156ms	remaining: 45.8s
17:	learn: 0.0988001	total: 158ms	remaining: 43.8s
18:	learn: 0.0960487	total: 168ms	remaining: 43.9s
19:	learn: 0.0918831	total: 176ms	re

In [19]:
print(r2_score(test_y, preds))
print((mean_squared_error(test_y, preds))**0.5)
print(mean_absolute_percentage_error(test_y, preds))

0.8037184813403366
5762295.386651807
0.06043976985745285


In [22]:
print(r2_score(test_y, preds))
print((mean_squared_error(test_y, preds))**0.5)
print(mean_absolute_percentage_error(test_y, preds))

0.841175148418048
5183403.7863689065
0.06435423471176022
